# 5 Useful Data Wrangling Techniques Using Python Pandas and data.world   
## Intro
### This notebook demonstrates a collection of data wrangling problems I frequently encounter, and my approaches to solving them.    
What this tutorial **is not**: 
* a comprehensive tutorial on data wrangling
* the only or even the best solutions    

What this tutorial **is**:
* fun and interesting (hopefully)
* things I encounter often in my work
* the best solutions I'm aware of (at this time)

### Agenda
1. Standardize a large, messy datetime column
2. Change time zones
3. "Standardizing" text (string) columns 
4. Multiprocessing
5. Reshaping dataframes with melt, pivot, & groupby   

### Dependencies     
* We'll be using the following Python packages: pandas, numpy, matplotlib, fuzzywuzzy, time, pytz, re, and multiprocessing.     
* The data is called using the datadotword Python package.    
  * Instructions for installing and configuring for the first time: https://github.com/datadotworld/data.world-py and https://data.world/nrippner/explore-the-data-world-python-sdk.            
  * If you installed datadotworld in the past, please ensure that you're using the latest version. In a terminal/command line enter:    
  `pip install --upgrade datadotworld`

### Import Packages

In [1]:
import pandas as pd               # data wrangling library
import datadotworld as ddw        # data.world SDK
import numpy as np                # data manipulation, math, linear algebra library
import matplotlib.pyplot as plt   # matlab style data visualization library
from fuzzywuzzy import process    # fuzzy string matching
import time                       # to measure processing time
import pytz                       # time zones
import re                         # Python regex
import multiprocessing as mp      # multicore distributed processing
from __future__ import print_function

%matplotlib inline

In [2]:
pd.set_option('display.max_rows', 40)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
#pd.set_option('max_colwidth', 70)

# Data Wrangling Techniques

## 1. Standardize a large, messy datetime column      
Note, hat tip to data.world user @hipplec for suggesting this technique on a [thread in data.world](https://data.world/databrett/white-house-visitors-2015-6/discuss/data-preparation/7533)     
For this example, we'll use some data I've hosted [here](https://data.world/nrippner/my-dataset) on data.world. The data table we're interested is a selection of three columns from the White House Visitor Logs dataset covering the years 2008 through 2016.     
![wrang_tut_1.png](https://data.world/api/nrippner/dataset/images/file/raw/wrang_tut_1.png)

We can see that the data file -- dates.csv -- has 3 columns and nearly 6 million rows.   
First, let's import the data from data.world. I'll demonstrate two methods to efficiently import large datasets from data.world.

#### 1. Copy and past URL     
Due to the large size (6 million rows), we may want to use the "Copy URL, Python/Pandas, or R code" approach to importing this data into Python (as opposed to writing a query) to avoid the compute time associated with querying over so many cells. 

![wrang_tut_2.png](https://data.world/api/nrippner/dataset/images/file/raw/wrang_tut_2.png)

In [3]:
# paste results:
dates_dataframe = pd.read_csv('https://query.data.world/s/hmbt8lm4cw6yj9vshigs178m')

#### 2. Use the `load_dataset` function in datadotworld python package      
This approach is highly optimized and convenient -- even with larger data files. Datasets are cached in a directory on your hard drive. 

In [4]:
dates = ddw.load_dataset('nrippner/my-dataset')
dates_dataframe = dates.dataframes['dates']

/Users/noahrippner/anaconda/envs/anaconda-py3/lib/python3.6/site-packages/datadotworld/datadotworld.py:194: UserWarning: You are using an outdated copy of nrippner/my-dataset. If you wish to use the latest version, call this function with the argument auto_update=True or force_update=True
  'force_update=True'.format(dataset_key))


In [5]:
print(dates_dataframe.shape)
dates_dataframe.head(3)

(5914766, 3)


,namefirst,namelast,appt_start_date
0,Stella,Adamopoulos,5/1/15
1,Muriel,Brosman,5/1/15
2,Avery,Brumfield,5/1/15


Our objective now is to convert the `appt_start_date column` to a Series of Pandas datetime objects. The column's dtype is currently numpy `object_` and the individual values are strings.

In [6]:
print(dates_dataframe.appt_start_date.dtype)
print(dates_dataframe.appt_start_date[0], type(dates_dataframe.appt_start_date[0]))

object
5/1/15 <class 'str'>


This seems like it should be relatively simple -- we could just pass the `df.APPT_START_DATE` to the `pandas.to_datetime()` function. Let's try it on a slice made up of the first five percent of the data.

In [7]:
start = time.time()
test = pd.to_datetime(dates_dataframe.appt_start_date[:295738])
end = time.time()
print("%.2f seconds to complete 1/20 of the rows" % (end-start))

28.65 seconds to complete 1/20 of the rows


In [8]:
print("~ %.1f minutes to complete all 5.9 million rows" % (((end-start) * 20) / 60.0))

~ 9.5 minutes to complete all 5.9 million rows


Processing 1/20 of the data on my machine took 44 seconds. At that rate, all the rows will take 15 minutes. Let's see if we can speed it up.     

In [9]:
print("Number of rows:", dates_dataframe.shape[0])
print("Number of unique dates:", len(dates_dataframe.appt_start_date.unique()))

start = time.time() 

def lookup(series):
    dates = {date:pd.to_datetime(date, errors='coerce') for date in series.unique()} 
    return series.map(dates)

dates_dataframe['NewDate'] = lookup(dates_dataframe.appt_start_date)

end = time.time()

print("%.1f seconds" % (end-start))
print("Couldn't be parsed:", sum(dates_dataframe.NewDate.isnull()) - sum(dates_dataframe.appt_start_date.isnull()) )   

Number of rows: 5914766
Number of unique dates: 158150
28.4 seconds
Couldn't be parsed: 162


In this example :
* we define a function called `lookup` which is intended to take as its argument a Pandas Series
* using `dictionary comprehension`, we iterate over each item in the set of unique values in our original series and map each to a `Pandas datetime` object created using the `to_datetime` function
* the `errors='coerce'` parameter tells the `to_datetime` function to return `np.nan` for values for which a datetime format cannot be inferred
* using the `pandas.Series.map` method, we return a new Series based on the key:value mappings in our dictionary comprehension.
* using this approach, we finished in 38 seconds -- a 96% reduction in processing time!     

In [10]:
dates_dataframe.head(3)

,namefirst,namelast,appt_start_date,NewDate
0,Stella,Adamopoulos,5/1/15,2015-05-01
1,Muriel,Brosman,5/1/15,2015-05-01
2,Avery,Brumfield,5/1/15,2015-05-01


In [11]:
dates_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5914766 entries, 0 to 5914765
Data columns (total 4 columns):
namefirst          object
namelast           object
appt_start_date    object
NewDate            datetime64[ns]
dtypes: datetime64[ns](1), object(3)
memory usage: 180.5+ MB


**Before moving on**, let's think about what caused Pandas `to_datetime` to take so long (17 minutes). Pandas `to_datetime` is processor intensive because it attempts to infer timestamp format, but it can be drastically sped up by including an additional argument: "_`format=`_".     

`new_series = pd.to_datetime(old_series, format="%Y/%m/%d")`     
 
By specifying the format via the `format=` parameter, the `to_datetime` algorithm skips the step of trying to infer the format, resulting in much shorter processing time. However, as we'll see, we could not use the format argument with this data.    

Let's take a closer look at the datetime column.

In [12]:
# helpful technique to look at composition of a column

# use apply method to change each value to type string
dates_dataframe['appt_start_date'] = dates_dataframe.appt_start_date.astype(str)

# dictionary comprehension to create list of the length of each datetime string
lengths = [len(j) for _, j in enumerate(dates_dataframe.appt_start_date.values)]

# convert to pandas Series in order to use the value_counts method
lengths = pd.Series(lengths).value_counts()

lengths

13    2058611
12    1351583
14    1339330
15     754861
11     258637
16     112105
8       13309
7        9488
10       8393
9        4049
6        1848
19       1639
20        581
18        325
3           4
17          2
5           1
dtype: int64

In [13]:
# function to return a sample datetime string for a given length
def inspect_date(length):
    return dates_dataframe[dates_dataframe.appt_start_date.str.len() == length]\
                                                     .appt_start_date.values[0]

# string formatting to format print output
print("{:<20} {:^12} {:^15}".format("Sample", "Length", "Count"))
print("-" * 53)

for i, j in zip(lengths.index, lengths.values):
    result = inspect_date(i)
    print("{:<20} {:>10} {:<2} {:>10} {:>7}".format(result, "length:", 
                                                    len(result), "count:", j))

Sample                  Length         Count     
-----------------------------------------------------
2/24/10 10:30           length: 13     count: 2058611
3/5/10 10:15            length: 12     count: 1351583
12/13/09 16:00          length: 14     count: 1339330
8/22/2010 12:30         length: 15     count:  754861
5/1/15 7:00             length: 11     count:  258637
12/12/2011 18:30        length: 16     count:  112105
10/22/10                length: 8      count:   13309
9/23/10                 length: 7      count:    9488
12/12/2011              length: 10     count:    8393
1/27/2011               length: 9      count:    4049
5/1/15                  length: 6      count:    1848
5/19/2009 7:00:00AM     length: 19     count:    1639
2/24/2009 11:30:00AM    length: 20     count:     581
9/24/20096:00:00PM      length: 18     count:     325
nan                     length: 3      count:       4
3/6/20099:30:00AM       length: 17     count:       2
41948                   length: 

**The take-away here** is that the datetime column lacks a consistent format. Some entries contain `hour:minute:seconds` and some don't. Sometimes the year is represented with 4 digits, sometimes only 2.    

When we specify a format using `"format="` in the function call, `to_datetime` throws an exception when encountering an unexpected type of datetime format. 

As far as I can tell our best bet (because of the lack of a consistent format) was the technique we used above, in which we used `to_datetime` on the set of unique values, and transformed the Series with the `map` method.

## 2. Changing time zones     
This is pretty straightforward, but worth looking at.    

First, let's download and import some timestamp data that has hour:minute:second.    

We'll use the `datadotworld.query` method to write a SQL query.

In [14]:
time_df = ddw.query('nrippner/datetime-sample',
                    'SELECT * FROM time_data LIMIT 1000').dataframe

time_df['created_date'] = pd.to_datetime(time_df.created_date)

* The `created_date` column (after feeding it to the `to_datetime` function) is a datetime64 object.  
* The individual objects that the `created_date` column comprises are pandas Timestamps.

In [15]:
time_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
id              1000 non-null int64
created_date    1000 non-null datetime64[ns]
dtypes: datetime64[ns](1), int64(1)
memory usage: 15.7 KB


In [16]:
item = time_df.created_date[0]
print(item)
print(type(item))

2017-02-18 08:05:01
<class 'pandas._libs.tslib.Timestamp'>


Let's imagine that we want to convert the data to Central time. By default, pandas Timestamp objects are _time zone agnostic_. Therefore, we have to ascertain the baseline timezone, because the data alone will not tell us. In this case, let's assume that I either contacted the database administrator or the database's documentation and learned that the time stamps were measured in UTC format (a common standard for databases).   
     
Knowing we're starting from UTC, here's how to go about changing the time zone in pandas. We'll use the `pytz` to get a list of time zones.

In [17]:
all_tz = pytz.all_timezones
#all_tz

In [18]:
time_df['central_tz'] = \
         time_df.created_date.apply(lambda x: x.tz_localize('UTC').tz_convert('US/Central'))

In [19]:
print("time zone:", time_df.central_tz[0].tz)
time_df.head()

time zone: US/Central


,id,created_date,central_tz
0,1164116,2017-02-18 08:05:01,2017-02-18 02:05:01-06:00
1,1164117,2017-02-18 08:05:05,2017-02-18 02:05:05-06:00
2,1164118,2017-02-18 08:05:06,2017-02-18 02:05:06-06:00
3,1164119,2017-02-18 08:05:09,2017-02-18 02:05:09-06:00
4,1164120,2017-02-18 08:05:10,2017-02-18 02:05:10-06:00


## 3. "Standardizing" text (string) columns      
In this section, we'll see how to correct discrepancies between two columns.     

> The typical use case is when you want to join two separate datasets, each with a common column to join on (for example, city name). More often than not we find spelling, syntax, or formatting differences between columns, which need to be brought into symmetry prior to executing a join.    

In this example, our goal is to join two datasets on a "Country" column. In this case, we're going to join a dataset on refugees by country with one for population by country, with the goal of creating a "Refugees Per Capita" feature.

In [20]:
refugee_df = ddw.query('nrippner/refugee-host-nations',
   ''' SELECT `unhcr_2015.csv/unhcr_2015`.`Refugees (incl. refugee-like situations)` as Refugees,
              `unhcr_2015.csv/unhcr_2015`.`Country / territory of asylum/residence` as Country
       FROM `unhcr_2015.csv/unhcr_2015` ''').dataframe

# filter out instances where Refugees = '*'
refugee_df = refugee_df[refugee_df.Refugees != '*']

# use pandas.to_numeric function to convert column to np.float64 dtype
refugee_df['Refugees'] = pd.to_numeric(refugee_df.Refugees)

# use pandas.DataFrame.groupby to get sum of refugees per country
refugee_df = refugee_df.groupby('Country', as_index=False).sum()

In [21]:
population_df = ddw.query('nrippner/refugee-host-nations',
    '''SELECT `worldbank_indicators.csv/worldbank_indicators`.`2015 [YR2015]` as Population_2015,
               `worldbank_indicators.csv/worldbank_indicators`.`Country Name` as Country
       FROM `worldbank_indicators.csv/worldbank_indicators`
       WHERE `worldbank_indicators.csv/worldbank_indicators`.`Series Name`
       LIKE "Population, total"''').dataframe


In [22]:
# show # of unique countries in each DataFrame
print("The refugee dataframe has %d unique Countries" % len(refugee_df.Country.unique()))
print("The population dataframe has %d unique Countries" % len(population_df.Country.unique()))

The refugee dataframe has 170 unique Countries
The population dataframe has 244 unique Countries


In [23]:
# In this step we use two Pandas.Series methods (Series.unique() and Series.isin()) to show the intersection between the two
# columns over the 'Country' column. 

A = sum(pd.Series(refugee_df.Country.unique()).isin(population_df.Country))
B = sum(pd.Series(population_df.Country.unique()).isin(refugee_df.Country))
print("%d out of %d unique countries in refugee_df are in population_df" 
                                   % (A, len(refugee_df.Country.unique())))
print("%d out of %d unique countries in population_df are in refugee_df" 
                                 % (B, len(population_df.Country.unique())))

143 out of 170 unique countries in refugee_df are in population_df
143 out of 244 unique countries in population_df are in refugee_df


The DataFrames share 143 unique countries. An inner merge at this point would result in a DataFrame 143 rows in length. We can probably do better than that.    
Take a look at the 27 countries in refugee_df that don't match

In [24]:
# use the .unique() method to give a numpy array of the unique countries.
# feed it to pd.Series() to convert it to a Pandas Series object so we can use
# the .isin() method
countries_unique = pd.Series(refugee_df.Country.unique())
countries_unique[~countries_unique.isin(population_df.Country)]
# (the ~ symbol before "countries" means "not")
# the items from refugee_df.Country that are not in population_df.Country

10                                       Bahamas
17              Bolivia (Plurinational State of)
30                          Central African Rep.
34                          China, Hong Kong SAR
36                                         Congo
41                                    Czech Rep.
42                                 Côte d'Ivoire
43                   Dem. People's Rep. of Korea
44                        Dem. Rep. of the Congo
47                                Dominican Rep.
49                                         Egypt
58                                        Gambia
73                        Iran (Islamic Rep. of)
84                                    Kyrgyzstan
100             Micronesia (Federated States of)
108                                        Nauru
126                                Rep. of Korea
127                              Rep. of Moldova
133        Serbia and Kosovo (S/RES/1244 (1999))
136                                     Slovakia
147                 

Now let's reconcile the country names between the two DataFrames wherever possible.  

In [25]:
# use list comprehension, the Python "enumerate" function, 
# and the pandas Series.isin() method to make list of non-matching countries
no_match = [[i, refugee_df.Country.unique()[i]] for i, j in 
     enumerate(pd.Series(refugee_df.Country.unique()).isin(population_df.Country)) if not j]

In [26]:
# a list of lists that pd.DataFrame will accept
no_match

[[10, 'Bahamas'],
 [17, 'Bolivia (Plurinational State of)'],
 [30, 'Central African Rep.'],
 [34, 'China, Hong Kong SAR'],
 [36, 'Congo'],
 [41, 'Czech Rep.'],
 [42, "Côte d'Ivoire"],
 [43, "Dem. People's Rep. of Korea"],
 [44, 'Dem. Rep. of the Congo'],
 [47, 'Dominican Rep.'],
 [49, 'Egypt'],
 [58, 'Gambia'],
 [73, 'Iran (Islamic Rep. of)'],
 [84, 'Kyrgyzstan'],
 [100, 'Micronesia (Federated States of)'],
 [108, 'Nauru'],
 [126, 'Rep. of Korea'],
 [127, 'Rep. of Moldova'],
 [133, 'Serbia and Kosovo (S/RES/1244 (1999))'],
 [136, 'Slovakia'],
 [147, 'Syrian Arab Rep.'],
 [150, 'The former Yugoslav Republic of Macedonia'],
 [161, 'United Rep. of Tanzania'],
 [162, 'United States of America'],
 [165, 'Venezuela (Bolivarian Republic of)'],
 [166, 'Viet Nam'],
 [167, 'Yemen']]

In [27]:
# create dataframe
no_match = pd.DataFrame(no_match, index=range(len(no_match)), columns=['idx', 'Country'])

In [28]:
# fuzzy matching, list comprehension, pd.Series.isin and enumerate to help find matches
# add results as a new column in our dataframe for easy viewing
no_match['Matches'] = [process.extractBests(j, population_df.Country.unique(), limit=3,
                        score_cutoff=70) for _, j in enumerate(no_match.Country.values)]

In [29]:
pd.options.display.max_colwidth = 80
no_match

,idx,Country,Matches
0,10,Bahamas,"[(Bahamas, The, 90)]"
1,17,Bolivia (Plurinational State of),"[(Bolivia, 90), (Isle of Man, 86), (Tonga, 72)]"
2,30,Central African Rep.,"[(Central African Republic, 88), (Congo, Rep., 86), (Korea, Rep., 86)]"
3,34,"China, Hong Kong SAR","[(Hong Kong SAR, China, 95), (China, 90), (Macao SAR, China, 71)]"
4,36,Congo,"[(Congo, Dem. Rep., 90), (Congo, Rep., 90), (Mongolia, 72)]"
5,41,Czech Rep.,"[(Czech Republic, 90), (Congo, Dem. Rep., 86), (Egypt, Arab Rep., 86)]"
6,42,Côte d'Ivoire,"[(Cote d'Ivoire, 96)]"
7,43,Dem. People's Rep. of Korea,"[(Korea, Dem. People’s Rep., 95), (Congo, Dem. Rep., 86), (Congo, Rep., 86)]"
8,44,Dem. Rep. of the Congo,"[(Congo, Dem. Rep., 95), (Bahamas, The, 86), (Congo, Rep., 86)]"
9,47,Dominican Rep.,"[(Dominica, 90), (Korea, Dem. People’s Rep., 86), (Dominican Republic, 84)]"


Most of these worked well, but for some reason Venezuela (line 24) and Kyrgyzstan (line 13) didn't properly yield a match (although they probably should have).    
We'll use pd.Series.str.contains() to manually take a look.

In [30]:
population_df[population_df.Country.str.contains('Venez')]

,Population_2015,Country
207,31108083.0,"Venezuela, RB"


* Now, we'll create a nested dictionary to input into the pd.DataFrame.replace function on the refugee_df dataframe.    
* The dictionary targets one column in the refugee_df dataframe: 'Country'.    
* The pd.DataFrame.replace function will look within the refugee_df.Country column, find each key in the lower nested dictionary, and replace them with their corresponding values.   

This process is (when possible), transforming the country names in refugee_df to match correctly with the same country's name in population_df.

In [31]:
repl = { 'Country' : {'Czech Rep.':'Czech Republic',
                      'Dominican Rep.':'Dominican Republic',
                      'Egypt':'Egypt, Arab Rep.',
                      'Micronesia (Federated States of)':'Micronesia, Fed. Sts.',
                      'Gambia':'Gambia, The',
                      'China, Hong Kong SAR':'Hong Kong SAR, China',
                      'Iran (Islamic Rep. of)':'Iran, Islamic Rep.',
                      'Kyrgyzstan':'Kyrgyz Republic',
                      'Rep. of Korea':'Korea, Rep.',
                      'Serbia and Kosovo (S/RES/1244 (1999))':'Serbia',
                      'Slovakia':'Slovak Republic',
                      'Syrian Arab Rep.':'Syrian Arab Republic',
                      'United Rep. of Tanzania':'Tanzania',
                      'United States of America':'United States',
                      'Venezuela (Bolivarian Republic of)':'Venezuela, RB',
                      'Viet Nam':'Vietnam',
                      'Yemen':'Yemen, Rep.',
                      'Bahamas':'Bahamas, The',
                      'Bolivia (Plurinational State of)':'Bolivia',
                      'Central African Rep.':'Central African Republic',
                      'CÃ´te d\'Ivoire':'Cote d\'Ivoire',
                      'Dem. Rep. of the Congo':'Congo, Dem. Rep.',
                      'Congo':'Congo, Rep.',
                      'Rep. of Moldova':'Moldova',
                      'The former Yugoslav Republic of Macedonia':'Macedonia, FYR',
                      'Dem. People\'s Rep. of Korea':'Korea, Dem. Peopleâs Rep.'
                     }
       }

refugee_df.replace(repl, inplace=True)

In [32]:
ref_unique = pd.Series(refugee_df.Country.unique())
a = 'count of number of non-matching country names'
print("{} {:_>10}".format(a, len(ref_unique[~ref_unique.isin(population_df.Country)])))
b = 'which country were we unable to match?'
print("{} {:_>21}".format(b, ref_unique[~ref_unique.isin(population_df.Country)].values[0]))


count of number of non-matching country names _________3
which country were we unable to match? ________Côte d'Ivoire


After reconciling the spelling differences, the only country that can't be matched
is Nauru. 
Now we have a matching column in each dataframe to join on.

In [33]:
# merge dataframes
refugee_df = refugee_df.merge(population_df, on='Country', how='inner')

# create new feature
refugee_df['Refugees_Per_Cap'] = refugee_df.Refugees / refugee_df.Population_2015

refugee_df.head()

,Country,Refugees,Population_2015,Refugees_Per_Cap
0,Afghanistan,257553,32526562.0,0.007918
1,Albania,79,2889167.0,0.000027
2,Algeria,94161,39666519.0,0.002374
3,Angola,15537,25021974.0,0.000621
4,Antigua and Barbuda,15,91818.0,0.000163


## 4. Multiprocessing      
Hat tip to [this blog](http://www.racketracer.com/2016/07/06/pandas-in-parallel/), where I discovered this technique.      
Sometimes despite our best efforts to write good Python/Pandas code, we have complex computations that we'd like to speed up. For processor-intensive jobs, multiprocessing is a solution.    

In [34]:
#import multiprocessor as mp

# first, define the function that you want to distribute across multiple CPU cores
# Note, this is a poorly written function, so we can play with multiprocessing
def func(s_split):
    
    def apply_func(x):
        count = 0
        for letter in list(x):
            for country in population_df.Country.values:
                if letter in list(country):
                    count += 1
        return count
    
    for n in range(500):
        out = s_split.apply(apply_func)
    return out

# multiprocessing
num_partitions = 10
num_cores = 6

def parallelize_series(s, func):
    s_split = np.array_split(s, num_partitions)
    pool = mp.Pool(num_cores)
    output = pd.concat(pool.map(func, s_split))
    pool.close()
    pool.join()
    return output

start = time.time()
func(refugee_df.Country)
end = time.time()
print("single job: %.2f" % (end - start))

start = time.time()
ref_unique['letter_counts'] = parallelize_series(refugee_df.Country, func)
end = time.time() 
print("multiprocessor: %.2f" % (end - start))

single job: 90.02
multiprocessor: 27.08


We see the improvement in processing time.   
The screenshot shows the jobs running concurrently on different cores of my quad-core laptop.

![wrang_tut_3.png](https://data.world/api/nrippner/dataset/images/file/raw/wrang_tut_3.png)

## 5. Reshaping dataframes with melt & pivot    

**transform from "wide" to "long" format using melt**    
For this exercise, we'll use the [Worldbank WDI Indicators data](https://data.world/worldbank/world-development-indicators) 

In [35]:
# population estimates by nation, by year
pop_dataframe = ddw.query(dataset_key='worldbank/world-development-indicators',
                          query = '''SELECT * FROM wdidata
                                    WHERE indicator_code = 'GC.TAX.TOTL.GD.ZS'
                                  ''').dataframe

In [36]:
print(pop_dataframe.shape)
pop_dataframe.head()

(264, 61)


,country_name,country_code,indicator_name,indicator_code,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,Belgium,BEL,Tax revenue (% of GDP),GC.TAX.TOTL.GD.ZS,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.485975,25.830408,26.347464,26.946863,26.797393,27.079800,26.640067,25.543235,24.949140,25.376233,25.353296,25.169655,24.710933,25.087498,23.620610,24.333897,24.711321,25.710013,26.210614,26.199248,24.634038,None
1,Belize,BLZ,Tax revenue (% of GDP),GC.TAX.TOTL.GD.ZS,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.406125,20.710974,19.935586,19.521906,19.916118,18.752931,19.538847,19.655231,19.698402,17.401511,17.827594,18.702015,19.299785,19.174321,19.663410,21.303913,21.784112,22.922524,21.699112,22.481123,23.596652,22.570675,22.457886,23.315711,23.390476,NaN,None
2,Benin,BEN,Tax revenue (% of GDP),GC.TAX.TOTL.GD.ZS,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.687221,14.412549,14.271349,14.635897,13.809283,14.213124,15.694394,16.115668,14.994099,15.533592,14.823271,14.413500,15.385296,NaN,NaN,None
3,Bermuda,BMU,Tax revenue (% of GDP),GC.TAX.TOTL.GD.ZS,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
4,Bhutan,BTN,Tax revenue (% of GDP),GC.TAX.TOTL.GD.ZS,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,5.429695,6.278335,7.208049,6.379292,6.753702,NaN,5.970228,5.191277,4.413943,4.952478,5.048773,6.233570,5.924630,6.611706,7.530245,6.538898,8.013738,7.015198,10.017850,8.527340,9.249166,9.365508,7.682697,8.346578,9.030991,7.519467,8.415932,9.235527,13.143297,13.507578,14.704148,14.377883,13.336893,NaN,None


In [37]:
id_vars = ['country_name', 'country_code', 'indicator_name', 'indicator_code']
pop_dataframe = pd.melt(pop_dataframe, 
                        id_vars=id_vars,
                        value_vars=pop_dataframe.columns[4:].values, 
                        var_name='Year', 
                        value_name='Tax_Rev_Percent_GDP')

In [38]:
print(pop_dataframe.shape)
pop_dataframe.head()

(15048, 6)


,country_name,country_code,indicator_name,indicator_code,Year,Tax_Rev_Percent_GDP
0,Belgium,BEL,Tax revenue (% of GDP),GC.TAX.TOTL.GD.ZS,1960,None
1,Belize,BLZ,Tax revenue (% of GDP),GC.TAX.TOTL.GD.ZS,1960,None
2,Benin,BEN,Tax revenue (% of GDP),GC.TAX.TOTL.GD.ZS,1960,None
3,Bermuda,BMU,Tax revenue (% of GDP),GC.TAX.TOTL.GD.ZS,1960,None
4,Bhutan,BTN,Tax revenue (% of GDP),GC.TAX.TOTL.GD.ZS,1960,None


**transform from "long" to "wide" using pivot_table**

In [39]:
# convert to numeric (errors='coerce' to convert strings to np.nan)
pop_dataframe['Tax_Rev_Percent_GDP'] = pd.to_numeric(pop_dataframe.Tax_Rev_Percent_GDP, 
                                                                       errors='coerce')
                     
# pivot from long to wide
index = ['country_name', 'country_code', 'indicator_name', 'indicator_code']
pop_dataframe = pd.pivot_table(pop_dataframe, 
                               values='Tax_Rev_Percent_GDP', 
                               columns=['Year'],
                               index=index,
                               aggfunc=np.mean)
# reset index
pop_dataframe.reset_index(inplace=True)

In [40]:
print(pop_dataframe.shape)
pop_dataframe.head()
# note that columns where all items were NaN were automatically dropped, 
# reducing the number of columns to 49.

(264, 49)


Year,country_name,country_code,indicator_name,indicator_code,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,Afghanistan,AFG,Tax revenue (% of GDP),GC.TAX.TOTL.GD.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.883364,5.229756,6.039428,8.434998,9.123651,8.854568,7.471639,7.158330,NaN,NaN,NaN
1,Albania,ALB,Tax revenue (% of GDP),GC.TAX.TOTL.GD.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.422768,8.553602,9.479293,13.701633,NaN,NaN,NaN,16.160843,16.790842,17.645359,NaN,NaN,NaN,NaN,NaN,NaN,18.022012,17.483234,16.497139,18.313552,18.510208,NaN
2,Algeria,DZA,Tax revenue (% of GDP),GC.TAX.TOTL.GD.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.693798,28.599517,30.607004,31.608446,25.026814,27.017746,36.928189,32.045511,31.524905,27.582053,26.803996,30.76547,40.750393,37.431012,45.252935,35.142903,34.402769,37.185853,NaN,NaN,NaN,NaN,NaN
3,American Samoa,ASM,Tax revenue (% of GDP),GC.TAX.TOTL.GD.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Andorra,AND,Tax revenue (% of GDP),GC.TAX.TOTL.GD.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Thanks for reading this notebook. I hope you enjoyed it!